In [30]:
import pandas as pd
import numpy as np
import pickle
from pathlib import Path
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import spearmanr
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns

In [31]:
INPUT_FILE = '../data/processed/f1_features_complete.csv'
OUTPUT_DIR = '../models'
MODEL_FILE = f'{OUTPUT_DIR}/xgboost_baseline4.2.pkl'
CONFIG_FILE = f'{OUTPUT_DIR}/config4.2.pkl'

Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

In [32]:
df = pd.read_csv(INPUT_FILE)

In [33]:
df = df.sort_values(['year', 'round', 'driver']).reset_index(drop=True)

In [34]:
df

,pct_puntos_actual,pct_linear_points,posicion_media,tendencia_ultimas_3,diff_con_lider_normalizada,progreso_temporada,year,round,driver,team,team_normalized,driver_quality_3y,team_avg_pos_3y,team_trend,pct_puntos_final
0,0.006601,0.047222,4.000000,4.000000,0.007151,0.055556,2008,1,ALO,Renault,Alpine,74.541311,6.373646,4.718750,0.084708
1,0.000000,0.000000,20.000000,20.000000,0.013751,0.055556,2008,1,BAR,Honda,Honda,43.745014,10.116858,4.840849,0.015952
2,0.003300,0.038889,7.000000,7.000000,0.010451,0.055556,2008,1,BOU,Toro Rosso,AlphaTauri,50.000000,13.121615,1.256303,0.007701
3,0.000000,0.000000,20.000000,20.000000,0.013751,0.055556,2008,1,BUT,Honda,Honda,64.062678,10.116858,4.840849,0.006051
4,0.000000,0.000000,20.000000,20.000000,0.013751,0.055556,2008,1,COU,Red Bull,Red Bull Racing,53.858974,9.260093,-1.383333,0.015402
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7574,0.121962,0.827273,4.454545,4.333333,0.043204,1.000000,2025,22,RUS,Mercedes,Mercedes,54.022727,5.935734,-0.030556,0.121962
7575,0.017552,0.406818,11.055556,11.666667,0.147615,1.000000,2025,22,SAI,Williams,Williams,56.522727,13.671389,0.661111,0.017552
7576,0.012601,0.340909,13.500000,14.000000,0.152565,1.000000,2025,22,STR,Aston Martin,Aston Martin,37.931818,10.215514,3.790670,0.012601
7577,0.009001,0.413636,12.333333,13.333333,0.156166,1.000000,2025,22,TSU,Red Bull Racing,Red Bull Racing,50.931818,4.455245,2.446290,0.009001


In [35]:
# Defino las features exactamente igual que en el modelo 4.1 para poder comparar
feature_cols = [
    'pct_puntos_actual',
    'pct_linear_points',
    'posicion_media',
    'tendencia_ultimas_3',
    'diff_con_lider_normalizada',
    'progreso_temporada',
    'driver_quality_3y',
    'team_avg_pos_3y',
    'team_trend'
]

target_col = 'pct_puntos_final'

# Separo X e y
X = df[feature_cols]
y = df[target_col]

# Guardo la columna 'year' aparte para poder imprimir qué años estoy usando en cada fold
years = df['year']

print(f"Features seleccionadas: {len(feature_cols)}")
print(f"Total de datos: {len(X)}")

Features seleccionadas: 9
Total de datos: 7579


In [36]:
from sklearn.model_selection import TimeSeriesSplit

# Configuro 5 divisiones. Esto creará ventanas expansivas de entrenamiento.
tscv = TimeSeriesSplit(n_splits=5)

# Uso los mismos parámetros que en el baseline
params = {
    'objective': 'reg:squarederror',
    'n_estimators': 200,
    'max_depth': 6,
    'learning_rate': 0.1,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'random_state': 42,
    'n_jobs': -1
}

# Listas para guardar los resultados de cada "examen" (fold)
mae_scores = []
rmse_scores = []
r2_scores = []
spearman_scores = []

In [37]:
fold = 1

# El método split de tscv nos devuelve los índices para cortar el dataframe
for train_index, test_index in tscv.split(X):
    
    # 1. Separar datos para este fold específico
    X_train_fold, X_test_fold = X.iloc[train_index], X.iloc[test_index]
    y_train_fold, y_test_fold = y.iloc[train_index], y.iloc[test_index]
    
    # Obtener qué años estamos usando (para visualizarlo en el print)
    train_years = years.iloc[train_index].unique()
    test_years = years.iloc[test_index].unique()
    
    # 2. Entrenar el modelo
    model = xgb.XGBRegressor(**params)
    model.fit(X_train_fold, y_train_fold)
    
    # 3. Predecir
    y_pred_fold = model.predict(X_test_fold)
    
    # 4. Calcular métricas
    mae = mean_absolute_error(y_test_fold, y_pred_fold)
    rmse = np.sqrt(mean_squared_error(y_test_fold, y_pred_fold))
    r2 = r2_score(y_test_fold, y_pred_fold)
    spearman, _ = spearmanr(y_test_fold, y_pred_fold)
    
    # Guardar métricas
    mae_scores.append(mae)
    rmse_scores.append(rmse)
    r2_scores.append(r2)
    spearman_scores.append(spearman)
    
    # 5. Imprimir reporte del fold
    print(f"\nFOLD {fold}:")
    print(f"  Entrenamiento: {min(train_years)} - {max(train_years)} ({len(X_train_fold)} muestras)")
    print(f"  Test (Validación): {min(test_years)} - {max(test_years)} ({len(X_test_fold)} muestras)")
    print(f"  Resultados -> MAE: {mae:.4f} | Spearman: {spearman:.4f} | R2: {r2:.4f}")
    print("-" * 50)
    
    fold += 1

# Promedios finales
print("\n" + "="*80)
print("RESULTADOS PROMEDIO (VALIDACIÓN CRUZADA)")
print("="*80)
print(f"MAE Promedio:      {np.mean(mae_scores):.4f} (+/- {np.std(mae_scores):.4f})")
print(f"RMSE Promedio:     {np.mean(rmse_scores):.4f}")
print(f"R² Promedio:       {np.mean(r2_scores):.4f}")
print(f"Spearman Promedio: {np.mean(spearman_scores):.4f}")


FOLD 1:
  Entrenamiento: 2008 - 2011 (1264 muestras)
  Test (Validación): 2011 - 2014 (1263 muestras)
  Resultados -> MAE: 0.0113 | Spearman: 0.9458 | R2: 0.8718
--------------------------------------------------

FOLD 2:
  Entrenamiento: 2008 - 2014 (2527 muestras)
  Test (Validación): 2014 - 2017 (1263 muestras)
  Resultados -> MAE: 0.0162 | Spearman: 0.8778 | R2: 0.8214
--------------------------------------------------

FOLD 3:
  Entrenamiento: 2008 - 2017 (3790 muestras)
  Test (Validación): 2017 - 2020 (1263 muestras)
  Resultados -> MAE: 0.0101 | Spearman: 0.8981 | R2: 0.9226
--------------------------------------------------

FOLD 4:
  Entrenamiento: 2008 - 2020 (5053 muestras)
  Test (Validación): 2020 - 2023 (1263 muestras)
  Resultados -> MAE: 0.0103 | Spearman: 0.9510 | R2: 0.9003
--------------------------------------------------

FOLD 5:
  Entrenamiento: 2008 - 2023 (6316 muestras)
  Test (Validación): 2023 - 2025 (1263 muestras)
  Resultados -> MAE: 0.0112 | Spearman: 0

In [ ]:
'''
Robustez Extrema: Hemos obtenido un Spearman promedio de 0.9141. 
Esto confirma que el modelo no solo predice bien la era actual (como vimos en el notebook 4.1), 
sino que es capaz de ordenar correctamente la clasificación final (ranking) consistentemente, independientemente de la temporada.

Precisión Quirúrgica: El MAE promedio de 0.0118 indica que el error de predicción es de apenas el 1.18% de los puntos totales. 
En una temporada estándar de 600 puntos, esto equivale a un margen de error de solo +/- 7 puntos.

Adaptabilidad Histórica: El análisis por folds demuestra que el modelo resiste los cambios de reglamento. 
Aunque hubo una ligera caída lógica en la transición a la era híbrida (Fold 2, 2014), el modelo mantuvo una correlación alta (>0.87), 
demostrando que las features de ingeniería (driver_quality, team_trend) aportan valor real más allá del coche de turno.

'''


''